# 06a. Working on a cluster - local part

## Overview

In this notebook and its remote counterpart `06b`, you will learn how to:

 - Synchronize deployments between the local machine and a notebook running on the cluster.
 - Perform simple computations using the Dask deployment on a deployed notebook.
 - Clear synchronized deployments.

## Import idact

It's recommended that *idact* is installed with *pip*. Alternatively, make sure the dependencies are installed: `pip install -r requirements.txt`, and add *idact* to path, for example:

In [1]:
import sys
sys.path.append('../')

We will use a wildcard import for convenience:

In [2]:
from idact import *
import bitmath

## Load the cluster

Let's load the environment and the cluster. Make sure to use your cluster name.

In [3]:
load_environment()
cluster = show_cluster("hpc")
cluster

Cluster(pro.cyfronet.pl, 22, plggarstka, auth=AuthMethod.PUBLIC_KEY, key='C:\\Users\\Maciej/.ssh\\id_rsa_6p', install_key=False, disable_sshd=False)

In [4]:
access_node = cluster.get_access_node()
access_node.connect()

## Allocate nodes, deploy Jupyter and Dask

We will be working with Dask on a Jupyter Notebook deployed on the cluster. Make sure to adjust `--account`, same as in previous notebooks 

In [5]:
nodes = cluster.allocate_nodes(nodes=3,
                               cores=2,
                               memory_per_node=bitmath.GiB(10),
                               walltime=Walltime(minutes=20),
                               native_args={
                                   '--account': 'intdata'
                               })
nodes

2018-11-24 15:15:57 INFO: Creating the ssh directory.


Nodes([Node(NotAllocated),Node(NotAllocated),Node(NotAllocated)], SlurmAllocation(job_id=14335314))

In [6]:
nodes.wait()
nodes

Nodes([Node(p0276:52339, 2018-11-24 14:36:03.264418+00:00),Node(p0281:56749, 2018-11-24 14:36:03.264418+00:00),Node(p0284:34760, 2018-11-24 14:36:03.264418+00:00)], SlurmAllocation(job_id=14335314))

Deploy a notebook:

In [7]:
nb = nodes[0].deploy_notebook()
nb

JupyterDeployment(8080 -> Node(p0276:52339, 2018-11-24 14:36:03.264418+00:00)

Deploy Dask on all three nodes:

In [8]:
dd = deploy_dask(nodes)
dd

2018-11-24 15:16:27 INFO: Deploying Dask on 3 nodes.
2018-11-24 15:16:27 INFO: Connecting to p0276:52339 (1/3).
2018-11-24 15:16:27 INFO: Connecting to p0281:56749 (2/3).
2018-11-24 15:16:29 INFO: Connecting to p0284:34760 (3/3).
2018-11-24 15:16:31 INFO: Deploying scheduler on the first node: p0276.
2018-11-24 15:16:43 INFO: Checking scheduler connectivity from p0276 (1/3).
2018-11-24 15:16:43 INFO: Checking scheduler connectivity from p0281 (2/3).
2018-11-24 15:16:43 INFO: Checking scheduler connectivity from p0284 (3/3).
2018-11-24 15:16:43 INFO: Deploying workers.
2018-11-24 15:16:43 INFO: Deploying worker 1/3.
2018-11-24 15:16:54 INFO: Deploying worker 2/3.
2018-11-24 15:17:05 INFO: Deploying worker 3/3.
2018-11-24 15:17:17 INFO: Validating worker 1/3.
2018-11-24 15:17:17 INFO: Validating worker 2/3.
2018-11-24 15:17:17 INFO: Validating worker 3/3.


DaskDeployment(scheduler=tcp://localhost:46388/tcp://172.20.65.21:46388, workers=3)

## Synchronize the deployments

It may be useful to access the allocated nodes, or any other deployment above from another notebook.

In particular, we need to access the Dask deployment from the notebook that was deployed on the cluster, in order to perform computations.

Synchronizing a deployment involves *pushing* it first, and the *pulling* on another notebook.

Let's push the allocation first:

In [9]:
cluster.push_deployment(nodes)

2018-11-24 15:17:17 INFO: Pushing deployment: Nodes([Node(p0276:52339, 2018-11-24 14:36:03.264418+00:00),Node(p0281:56749, 2018-11-24 14:36:03.264418+00:00),Node(p0284:34760, 2018-11-24 14:36:03.264418+00:00)], SlurmAllocation(job_id=14335314))


Then Jupyter:

In [10]:
cluster.push_deployment(nb)

2018-11-24 15:17:25 INFO: Pushing deployment: JupyterDeployment(8080 -> Node(p0276:52339, 2018-11-24 14:36:03.264418+00:00)


And finally Dask:

In [11]:
cluster.push_deployment(dd)

2018-11-24 15:17:33 INFO: Pushing deployment: DaskDeployment(scheduler=tcp://localhost:46388/tcp://172.20.65.21:46388, workers=3)


We will pull the deployments on the remote notebook in a moment.

## Copy notebook `06b` to the cluster

Drag and drop `06b-Working_on_a_cluster_-_remote_part.ipynb` to the deployed notebook, and open it there.

In [12]:
nb.open_in_browser()

## Follow the instructions in notebook `06b`

Follow the instructions until you are referred back to this notebook.

## Examine Dask Dashboards

You can always take a look how your computations look on the dashboards:

In [13]:
client = dd.get_client()
client

Client Scheduler: tcp://localhost:46388 Dashboard: http://localhost:47991/status,Cluster Workers: 3 Cores: 6 Memory: 32.21 GB


In [14]:
dd.diagnostics.open_all()

In [15]:
client.close()

## Clear synchronized deployments

Deployments are cleared automatically if they are expired or cancelled. They can also be cleared manually by  running:

In [16]:
cluster.clear_pushed_deployments()

2018-11-24 15:20:11 INFO: Clearing deployments.


## Cancel Dask and Jupyter deployments (optional)

In [17]:
nb.cancel()

2018-11-24 15:20:13 INFO: Cancelling Jupyter deployment.


In [18]:
dd.cancel()

2018-11-24 15:20:20 INFO: Cancelling worker deployment on p0284.
2018-11-24 15:20:26 INFO: Cancelling worker deployment on p0281.
2018-11-24 15:20:33 INFO: Cancelling worker deployment on p0276.
2018-11-24 15:20:40 INFO: Cancelling scheduler deployment on p0276.


## Cancel the allocation

It's important to cancel an allocation if you're done with it early, in order to minimize the CPU time you are charged for.

In [19]:
nodes.running()

True

In [20]:
nodes.cancel()

2018-11-24 15:20:49 INFO: Cancelling job 14335314.


In [21]:
nodes.running()

False

## Next notebook

In the next notebook we will take a look on how to adjust deployment timeouts.